In [352]:
import numpy as np
import pandas as pd
import datetime as dt
import requests
import sqlite3
import json
from bs4 import BeautifulSoup
import os
import time

In [348]:
class Animal():
    def __init__(self, vars ,**kwargs):
        self.__dict__.update(vars, **kwargs)

class crawl(Animal):

    def ptt(self, url):
        header = self.__dict__
        r = requests.get(url, headers=header)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser")
       
            web_list = []
            for i in soup.find_all("div", class_='r-ent'):
                j = i.find_all("a", href=True)
                if len(j)>0:
                    web_list.append("https://www.ptt.cc/"+j[0]["href"])

        else: return "something wrong about ptt stock %s." %url

        for j in web_list:
            r2 = requests.get(j, headers=header)
            if r2.status_code == 200:

                soup2 = BeautifulSoup(r2.text, "html.parser")
                soup2.title.string.split("-")[0]

                text = soup2.find(id="main-container").text.split("--")
                pre_text = text[0].split('\n')

                push_text= []
                for i in soup2.find_all("div", class_="push"):
                    push_text.append(i.text)
                
                s = soup2.title.string.split("-")[0]

                with open(os.path.join("news/ptt/", repr(s).replace("\\","").replace("/","").replace(" ","")+".txt"), "w") as f:
                    f.write("\n".join(pre_text[1:]))
                    f.write("\n".join(push_text))

            else: return "th post %s has something wrong" %j
    
    def udn(self, url):
        header = self.__dict__
        r = requests.get(url, headers=header)

        if r.status_code == 200:
            soup = BeautifulSoup(r.text, "html.parser")
            
            web_list = []
            for i in soup.find_all("div", class_="story__content"):
                j = i.find_all("a", href=True)
                web_list.append(j[0]["href"])

        else: return "something wrong about udn %s." %url

        for j in web_list:
            r2 = requests.get(j, headers=header)
            if r2.status_code == 200:

                soup = BeautifulSoup(r2.text, "html.parser")

                timing = soup.find("time",class_="article-body__time")
                if timing is not None:
                    
                    timing = timing.text
                    title = soup.find(id="story_art_title").text
                    body = soup.find("section",class_="article-body__editor").text

                    name = repr(title).replace("\\","").replace("/","").replace(" ","")

                    with open(os.path.join("news/udn/", name+".txt"), "w") as f:
                        f.write(title)
                        f.write('\n')
                        f.write(timing)
                        f.write(body)

            else: return "th post %s has something wrong" %j



Set up the header for crawling

In [240]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
cookie = "uuid_tt_dd=10_20564905260-1703225841522-750867; c_first_ref=www.google.com; c_segment=3; c_adb=1; dc_sid=df9d26e5f60c9796335d2b2bdc46e0aa; hide_login=1; SidecHatdocDescBoxNum=true; creative_btn_mp=3; dc_session_id=10_1704181968889.661952; SESSION=ee7d1139-efc1-4491-8412-ffed93215f66; c_first_page=https%3A//blog.csdn.net/yinjun3215/article/details/108477055; c_dsid=11_1704184443684.220868; c_pref=https%3A//www.google.com/; c_ref=https%3A//blog.csdn.net/yinjun3215/article/details/108477055; c_utm_medium=distribute.pc_relevant.none-task-blog-2%7Edefault%7ECTRLIST%7ERate-1-102261567-blog-108477055.235%5Ev40%5Epc_relevant_anti_t3; c_utm_relevant_index=2; c_page_id=default; log_Id_pv=19; loginbox_strategy=%7B%22taskId%22%3A308%2C%22abCheckTime%22%3A1704163911128%2C%22version%22%3A%22exp1%22%2C%22blog-threeH-dialogtipShowTimes%22%3A8%2C%22blog-threeH-dialog%22%3A1704182063207%7D; log_Id_view=477; log_Id_click=7; dc_tos=s6mm2j"
x = dict({"User-Agent":user_agent, "Cookie":cookie})

In [350]:
test = crawl(x)

In [283]:
for k in range(6805, 6819+1):
    ptt_path = "https://www.ptt.cc/bbs/Stock/index"+str(k)+".html"
    test.ptt(url=ptt_path)
    if (k%100 == 0):
        print(k)
    time.sleep(5)

In [ ]:
for k in range(75,90):
    ptt_path = "https://money.udn.com/rank/ajax_newest/1001/0/"+str(k)
    test.udn(url=ptt_path)
    if (k%2 == 0):
        print(k)
    time.sleep(5)

In [399]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
cookie="sDataFile=CAR%2DY%2CSTN%2DN%2CICM%2DY%2CICD%2DY%2CEVF%2DN%2CSVR%2DY%2CCMP%2DY%2CCSE%2DN%2CIOT%2DN%2CSMG%2DN%2CBWC%2DN%2CSML%2DN%2CSMF%2DY%2CATC%2DN%2CAAI%2DN%2CDHM%2DN%2CMCN%2DY%2CGNT%2DN%2CAIP%2DN%2CAIR%2DN%2C5GF%2DN%2CPCE%2DN%2C;"
referer = "https://www.digitimes.com.tw/tech/dt/allnewslist.asp?CnlID=99"
content_type = "application/x-www-form-urlencoded; charset=UTF-8"
x = dict({"User-Agent":user_agent, "Cookie":cookie, "Referer":referer, "Content-Type":content_type})

In [400]:
r = requests.get("https://www.digitimes.com.tw/tech/ajax/newslist_ajax.asp", headers=x, auth=("james.jk.huang@foxconn.com", "4GJ0L48X27ZQHP0E5KN6B3UZ1HF3D1NL0PQIYV3CF3YOWL3L"))
r.status_code

200

In [401]:
soup = BeautifulSoup(r.text, "html.parser")

In [402]:
soup.find_all("div",class_="subject_area")

[<div class="col-md-7 col-sm-7 col-xs-8 subject_area" hb="1"><a href="/tech/dt/n/shwnws.asp?cnlid=1&amp;cat=90&amp;id=0000682427_XSD5KRPE17VM7B8538EST" target="_blank"><p style="margin:5px 0;"><span>比亞迪單季純電動車銷量超越Tesla　4Q23成全球龍頭</span>   <span class="visible-xs">1/3 12:27</span></p></a></div>,
 <div class="col-md-7 col-sm-7 col-xs-8 subject_area" hb="1"><a href="/tech/dt/n/shwnws.asp?cnlid=1&amp;cat=70&amp;id=0000682446_D8J3EBWF5H3S9F5VRSZ3R" target="_blank"><p style="margin:5px 0;"><span>2023年高階手機市場持續擴張　蘋果強碰三星、華為</span>   <span class="visible-xs">1/3 12:14</span></p></a></div>,
 <div class="col-md-7 col-sm-7 col-xs-8 subject_area" hb="1"><a href="/tech/dt/n/shwnws.asp?cnlid=1&amp;cat=40&amp;id=0000682429_RNJ5PERJ1X64JZL6W3XPG" target="_blank"><p style="margin:5px 0;"><span>鞏固HBM領導地位　SK海力士ㅡM15X新廠將生產HBM</span>   <span class="visible-xs">1/3 12:06</span></p></a></div>,
 <div class="col-md-7 col-sm-7 col-xs-8 subject_area" hb="1"><a href="/tech/dt/n/shwnws.asp?cnlid=9&amp;cat=400&amp;id=00

In [381]:
r.text

'<div class="col-md-12 col-sm-12 col-xs-12" id="datelist"><span style="float:right;font-size:14px;"><input type="text" class="showdate" name="sdate" maxlength="10" ReadOnly  placeholder="查詢其他日期" ID="sdate" value="2024-01-03" style="width:130px;"></span></div><div class="col-md-12 col-sm-12 col-xs-12 m-b-5" style="padding: 0;margin-left: 8px;">\t<div class="col-md-12 col-sm-12 col-xs-12" style="background-color:#fef9f4;border-top:1px solid #fcd5b5;border-bottom:1px solid #fcd5b5;font-weight:bold;padding:5px 0;">\t\t<div class="col-md-12 col-sm-12 col-xs-12" style="text-align:right;">\t\t\t&nbsp;&nbsp;排序方式：\t\t\t<a href="javascript: void(0);" onclick="filtercat2(\'0\',\'\',\'0\',\'0\');">時間▼</a>&nbsp;&nbsp;&nbsp;<a href="javascript: void(0);" onclick="filtercat2(\'0\',\'\',\'1\',\'0\');">分類</a>&nbsp;&nbsp;&nbsp;<a href="javascript: void(0);" onclick="filtercat2(\'0\',\'\',\'2\',\'0\');">熱門度</a>\t\t</div>\t</div></div><div class="col-md-12 col-sm-12 col-xs-12" id="ajaxcontainer" style="pa